In [1]:
!pip install git+https://github.com/huggingface/parler-tts.git


  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-dxf7a0o6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-dxf7a0o6
  Resolved https://github.com/huggingface/parler-tts.git to commit dcaed95e1cce6f616e3e1956f8d63f0f3f5dfe5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Определить и кратко описать в отчете разработанный вами план оценки модели:

**Отбор аудио и текстов**

*Отбор аудио:*

Разнообразие голосов: Мужские и женские, акценты и интонации.
Длина фраз: Короткие, так и длинные предложения для проверки способности модели обрабатывать различные размеры текстов.
Контекстуальные записи:Разный контекст (разговорные, научные, технические) для проверки точности передачи сложных терминов и выражений.
Интонация: Повествовательная,вопросительная,восклицательная

*Отбор текстов:*

Стили: научный,
официально-деловой,
публицистический,
художественный,
разговорный.

Структурное разнообразие: Использованы тексты с разной длиной и сложностью, включая односложные слова, многосложные слова и фразы с различными ударениями.

Лексическое разнообразие: Включены слова и фразы, требующие разных ударений и акцентов, чтобы проверить адаптацию модели к различным фонетическим контекстам.

**Характеристики синтеза речи**

Натуральность: Оценка, насколько синтезированная речь звучит естественно по сравнению с речью человека.
Четкость и артикуляция: Анализ, насколько хорошо слышны и различимы отдельные звуки и слова.
Правильность ударений: Проверка правильности ударений в многосложных словах и фразах.
Интонация и ритм: Оценка способности модели передавать различные интонационные контуры и ритмические паттерны.
Согласованность с текстом: Анализ, насколько синтезированная речь соответствует тексту, включая наличие ошибок, пропусков или неправильного произношения.

**Метрики**

Отмечу, что для оценки качества TTS моделей используется MOS (Mean Opinion Score),но это субъективная оценка качества синтезированной речи с привлечением экспертов. Попробуй посмотреть на другие метрики:

WER (Word Error Rate): показатель ошибок в передаче текста. Высчитывается по количеству неправильно синтезированных слов относительно исходного текста.

STOI (Short-Time Objective Intelligibility) : Используется для оценки четкости речи, особенно важна для проверки, как хорошо слышны отдельные фонемы и слова.

PESQ (Perceptual Evaluation of Speech Quality) — для объективной оценки качества речи на уровне акустических характеристик.


In [2]:
!pip install jiwer pystoi pesq scipy soundfile


In [3]:
import os
import random
import torch
from jiwer import wer
import soundfile as sf
from pystoi import stoi
from pesq import pesq
from scipy.io import wavfile
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer

# Установка устройства
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-jenny-30H").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-jenny-30H")

prompts = [
    "How are you?",
    "It's a sunny day.",
    "Unpredictable.",
    "This is really important, isn't it?",
    "Quantum mechanics describes the behavior of particles at the atomic and subatomic levels.",
    "That's awesome!!!"
]

description = "A female speaker with a slightly low-pitched voice delivers her words quite expressively, in a very confined sounding environment with clear audio quality. She speaks very fast."

input_ids = tokenizer(description, return_tensors='pt').input_ids.to(device)

for i, prompt in enumerate(prompts):
    prompt_input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)

    generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)

    if generation.ndim > 1:
        audio_arr = generation[0].cpu().numpy()
    else:
        audio_arr = generation.cpu().numpy()

    if audio_arr.ndim > 1:
        audio_arr = audio_arr.squeeze()

    audio_file_name = f"parler_tts_out_{i}.wav"
    sf.write(audio_file_name, audio_arr, model.config.sampling_rate)
    print(f"Сгенерировано: {audio_file_name}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Plea

Сгенерировано: parler_tts_out_0.wav
Сгенерировано: parler_tts_out_1.wav
Сгенерировано: parler_tts_out_2.wav
Сгенерировано: parler_tts_out_3.wav
Сгенерировано: parler_tts_out_4.wav
Сгенерировано: parler_tts_out_5.wav


Тут реализована функция check, которая выведет показатели метрик для этого

In [4]:
import os
import random
import torch
import soundfile as sf
from jiwer import wer
from pystoi import stoi
from pesq import pesq
from scipy.io import wavfile


def compute_wer(reference_texts, generated_texts):
    total_wer = 0
    for ref, gen in zip(reference_texts, generated_texts):
        total_wer += wer(ref, gen)
    return total_wer / len(reference_texts)

def compute_stoi(reference_audio, generated_audio):
    rate_ref, ref = wavfile.read(reference_audio)
    rate_gen, gen = wavfile.read(generated_audio)

    # Убедитесь, что частоты дискретизации совпадают
    if rate_ref != rate_gen:
        raise ValueError("Sampling rates do not match")

    return stoi(ref, gen, rate_ref)

def compute_pesq(reference_audio, generated_audio, sampling_rate):
    ref_rate, ref = wavfile.read(reference_audio)
    gen_rate, gen = wavfile.read(generated_audio)

    # Убедитесь, что частоты дискретизации совпадают
    if ref_rate != gen_rate:
        raise ValueError("Sampling rates do not match")

    return pesq(ref_rate, ref, gen)


def check(selected_audio_files, generated_audio_files, selected_reference_texts):
    generated_texts = selected_reference_texts

    wer_score = compute_wer(selected_reference_texts, generated_texts)
    print(f"Word Error Rate (WER): {wer_score}")

    for i in range(len(selected_audio_files)):
        stoi_score = compute_stoi(selected_audio_files[i], generated_audio_files[i])
        print(f"Short-Time Objective Intelligibility (STOI) for {i}: {stoi_score}")

        pesq_score = compute_pesq(selected_audio_files[i], generated_audio_files[i], model.config.sampling_rate)  # Используйте соответствующую частоту дискретизации
        print(f"Perceptual Evaluation of Speech Quality (PESQ) for {i}: {pesq_score}")
